In [1]:
import json
import re
from _pipeline import create_payload, model_req  # Import AI model interaction functions

# Load previous best prompts (Reinforcement Learning Feedback)
try:
    with open("best_prompts.json", "r") as file:
        best_prompt_history = json.load(file)  # Load previous best prompts and scores
except FileNotFoundError:
    best_prompt_history = {}

important_terms = [
    "functional requirements", "non-functional requirements", "real-time study support",
    "personalized tutoring", "exam preparation"
]

def score_response(response):
    # Basic score from word count & keyword match
    score = len(response.split())
    score += sum(1 for term in important_terms if term in response.lower()) * 10

    # AI-based critique: Clarity, Depth, and Accuracy
    critic_prompt = f"""
    Evaluate the following requirement analysis response based on these criteria:
    - **Clarity**: Is the response well-structured and understandable?
    - **Depth**: Does it cover both functional and non-functional requirements in detail?
    - **Accuracy**: Are the statements factually correct and aligned with best practices?

    Score the response on a scale of 1-10 and provide a brief explanation.

    Response:
    {response}

    Format: "Score: X/10. Reason: ..."
    """

    # Call the AI model to critique its own response
    payload_critic = create_payload(
        target="ollama",
        model="llama3.2:latest",
        prompt=critic_prompt,
        temperature=0.5  # More deterministic
    )
    
    _, critic_feedback = model_req(payload_critic)  # Get AI's evaluation
    
    # Extract the numeric score from the AI's feedback
    match = re.search(r"Score:\s*(\d+)", critic_feedback)
    critic_score = int(match.group(1)) if match else 0  # Default to 0 if no match found

    # Final combined score
    final_score = score + (critic_score * 10)  # Weight the AI's evaluation

    return final_score, critic_feedback  # Return both score & AI’s reasoning

#### (1) AI Generates a Structured Prompt for Requirement Analysis
meta_prompt = (
    "Create a highly structured prompt that extracts detailed functional and non-functional "
    "requirements for an AI-powered Study Companion Discord Bot. The structured prompt should ensure "
    "clarity, completeness, and depth in the generated response."
)

payload_meta = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=meta_prompt,
    temperature=0.7
)
_, generated_prompt = model_req(payload_meta)

#### (2) AI Executes the Generated Prompt
payload_exec = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=generated_prompt,
    temperature=1.0,
    num_ctx=4096,
    num_predict=2048
)
_, initial_response = model_req(payload_exec)

#### (3) AI Evaluates & Suggests Improvements
payload_eval = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=f"Analyze and suggest improvements: {initial_response}",
    temperature=0.7
)
_, improvement_suggestions = model_req(payload_eval)

#### (4) AI Generates an Improved Prompt
payload_improve = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=f"Revise the prompt based on these improvements: {improvement_suggestions}",
    temperature=0.7
)
_, improved_prompt = model_req(payload_improve)

#### (5) AI Executes the Improved Prompt
payload_final_exec = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=improved_prompt,
    temperature=1.0,
    num_ctx=4096,
    num_predict=2048
)
_, final_response = model_req(payload_final_exec)

# Compare responses using AI Critic Feedback
i_score, i_critique = score_response(initial_response)
f_score, f_critique = score_response(final_response)

best_prompt = improved_prompt if f_score > i_score else generated_prompt
best_response = final_response if f_score > i_score else initial_response
best_score = max(i_score, f_score)

# Save the Best Performing Prompt
best_prompt_history[best_prompt] = {"feedback_score": best_score}
with open("best_prompts.json", "w") as file:
    json.dump(best_prompt_history, file, indent=4)

# Output Results
print(f"✅ Best Optimized Prompt: {best_prompt}")
print(f"📜 Response: {best_response}")
print(f"🏆 Score: {best_score}")
print(f"🔍 Initial Critique: {i_critique}")
print(f"🔍 Final Critique: {f_critique}")


{'model': 'llama3.2:latest', 'prompt': 'Create a highly structured prompt that extracts detailed functional and non-functional requirements for an AI-powered Study Companion Discord Bot. The structured prompt should ensure clarity, completeness, and depth in the generated response.', 'stream': False, 'options': {'temperature': 0.7}}
{'model': 'llama3.2:latest', 'prompt': "**AI-Powered Study Companion Discord Bot Functional Requirements**\n\n**Functional Requirements:**\n\n1. **User Registration and Authentication**\n\t* Users must be able to create a unique account using Discord's authentication system.\n\t* The bot must store user information securely, including username, email, and password hash.\n\t* Users must be able to log in using their registered credentials.\n2. **Study Plan Generation**\n\t* The bot must generate personalized study plans for each user based on their course schedule, deadlines, and learning style.\n\t* Study plans should include a list of topics to cover, rele